In [1]:
import requests
import json

elasticSearchUrl = "http://localhost:9200"

In [18]:
"""
tie_breaker - от 0 до 1 - увеличивает релевантность ответа 
если текст запроса содержиться сразу в нескольких полях.
По умолчанию = 0

best_fields - для определения релевантности используется оценка лучше всего совпадающего поля
"""


usersSearch = 'star'

search = json.dumps(
    {
    "query": {
        "multi_match" : {
              "query": usersSearch, 
              "fields": ["title", "overview"],
            'type': 'best_fields',
            'tie_breaker': 0.5
            }
          },
        'size': '100',
        'explain': True
        }
)

resp = requests.get("http://localhost:9200/tmdb/_doc/_search?pretty", 
                   data=search,
                   headers={"Content-Type": "application/json"}).json()

for idx, hit in enumerate(resp['hits']['hits']):
    print(idx, '\t', hit['_score'], '\t', hit['_source']['title'])

0 	 4.5374618 	 Cars 2
1 	 4.411681 	 Star Wars: Episode VI - Return of the Jedi
2 	 4.078724 	 Kick-Ass 2
3 	 4.008845 	 Star Trek
4 	 3.9586582 	 Amélie
5 	 3.8091507 	 The Truman Show
6 	 3.670525 	 Ted
7 	 3.541635 	 The Bourne Legacy
8 	 3.3871846 	 Star Trek Into Darkness
9 	 3.3871846 	 The Fault in Our Stars
10 	 2.31173 	 Star Wars: Episode IV - A New Hope
11 	 2.31173 	 Star Wars: Episode I - The Phantom Menace
12 	 2.31173 	 Star Wars: Episode III - Revenge of the Sith
13 	 2.31173 	 Star Wars: Episode II - Attack of the Clones
14 	 2.0904822 	 Star Wars: Episode V - The Empire Strikes Back


In [19]:
"""

tie_breaker - от 0 до 1 - увеличивает релевантность ответа 
если текст запроса содержиться сразу в нескольких полях.
По умолчанию = 0

most_fields - для определения релевантности используется суммарная оценка релевантности по всем поля
"""


usersSearch = 'star'

search = json.dumps(
    {
    "query": {
        "multi_match" : {
              "query": usersSearch, 
              "fields": ["title", "overview"],
            'type': 'most_fields',
            'tie_breaker': 0.5
            }
          },
        'size': '100',
        'explain': True
        }
)

resp = requests.get("http://localhost:9200/tmdb/_doc/_search?pretty", 
                   data=search,
                   headers={"Content-Type": "application/json"}).json()

for idx, hit in enumerate(resp['hits']['hits']):
    print(idx, '\t', hit['_score'], '\t', hit['_source']['title'])

0 	 4.5374618 	 Cars 2
1 	 4.411681 	 Star Wars: Episode VI - Return of the Jedi
2 	 4.078724 	 Kick-Ass 2
3 	 4.008845 	 Star Trek
4 	 3.9586582 	 Amélie
5 	 3.8091507 	 The Truman Show
6 	 3.670525 	 Ted
7 	 3.541635 	 The Bourne Legacy
8 	 3.3871846 	 Star Trek Into Darkness
9 	 3.3871846 	 The Fault in Our Stars
10 	 2.31173 	 Star Wars: Episode IV - A New Hope
11 	 2.31173 	 Star Wars: Episode I - The Phantom Menace
12 	 2.31173 	 Star Wars: Episode III - Revenge of the Sith
13 	 2.31173 	 Star Wars: Episode II - Attack of the Clones
14 	 2.0904822 	 Star Wars: Episode V - The Empire Strikes Back


In [23]:
"""

tie_breaker - от 0 до 1 - увеличивает релевантность ответа 
если текст запроса содержиться сразу в нескольких полях.
По умолчанию = 0

cross_fields - для определения релевантности объединяем поля в которых изщем и высчитываем общую релевантность
"""


usersSearch = 'star'

search = json.dumps(
    {
    "query": {
        "multi_match" : {
              "query": usersSearch, 
              "fields": ["title", "overview"],
            'type': 'cross_fields',
            'tie_breaker': 0.5
            }
          },
        'size': '100',
        'explain': True
        }
)

resp = requests.get("http://localhost:9200/tmdb/_doc/_search?pretty", 
                   data=search,
                   headers={"Content-Type": "application/json"}).json()

for idx, hit in enumerate(resp['hits']['hits']):
    print(idx, '\t', hit['_score'], '\t', hit['_source']['title'])

0 	 4.1537704 	 Cars 2
1 	 4.136367 	 Star Wars: Episode VI - Return of the Jedi
2 	 4.008845 	 Star Trek
3 	 3.7338238 	 Kick-Ass 2
4 	 3.623911 	 Amélie
5 	 3.4870458 	 The Truman Show
6 	 3.3871846 	 Star Trek Into Darkness
7 	 3.3871846 	 The Fault in Our Stars
8 	 3.3601427 	 Ted
9 	 3.2421517 	 The Bourne Legacy
10 	 2.31173 	 Star Wars: Episode IV - A New Hope
11 	 2.31173 	 Star Wars: Episode I - The Phantom Menace
12 	 2.31173 	 Star Wars: Episode III - Revenge of the Sith
13 	 2.31173 	 Star Wars: Episode II - Attack of the Clones
14 	 2.0904822 	 Star Wars: Episode V - The Empire Strikes Back


In [26]:
"""

bool - запрос аналогичный AND - выполняет сразу нгесколько условий

{
   "query": {
     "bool": {
       "must": [],
       "must_not": [],
       "should": [],
       "filter": []
     }
   }
 }
 
 must часть запроса содержит все запросы and. 
 must_not раздел содержит все not запросы. 
 should раздел содержит or запросы.
 filter содержит запросы на соответствие, но без учета ранжирования
 
 https://codedzen.ru/elasticsearch-urok-6-3-poisk/
"""

# WHERE CONCAT(overview, title) like '%star%' OR title like '%star trek%'

usersSearch = 'star'

search = json.dumps(
    {
    'query': {
        'bool': {
            'should': [
            {'multi_match': { 
               'query': usersSearch, 
                'fields': ['overview', 'title'],
                'type': 'cross_fields'                                
             }},
            { 
               'match_phrase': {
                    'title': {
                        'query': 'star trek',
                    }
                }
            }
            ]
        }
    },
    'size': 10,
    'explain': True
}
)

resp = requests.get("http://localhost:9200/tmdb/_doc/_search?pretty", 
                   data=search,
                   headers={"Content-Type": "application/json"}).json()

for idx, hit in enumerate(resp['hits']['hits']):
    print(idx, '\t', hit['_score'], '\t', hit['_source']['title'])

0 	 13.456478 	 Star Trek
1 	 11.369753 	 Star Trek Into Darkness
2 	 4.1537704 	 Cars 2
3 	 3.7338238 	 Kick-Ass 2
4 	 3.623911 	 Amélie
5 	 3.4870458 	 The Truman Show
6 	 3.3871846 	 The Fault in Our Stars
7 	 3.3601427 	 Ted
8 	 3.2421517 	 The Bourne Legacy
9 	 2.9805017 	 Star Wars: Episode VI - Return of the Jedi


In [48]:
# filter - принудительно оставляет в выыоде только определенные поля игнорируя их релевантность

usersSearch = 'star'

search = json.dumps(
    {
    'query': {
        'bool': {
            'should': [
            {'multi_match': { 
               'query': usersSearch, 
                'fields': ['overview', 'title'],
                'type': 'cross_fields',
                'boost': 5 # Усилили условие в 5 раз
             }},
            { 
               'match_phrase': {
                    'title': {
                        'query': 'star trek',
                        'boost': 0.1 # Ослабили условие в 10 раз
                    }
                }
            }
            ],
            'filter': {
                'match_phrase': {
                        'title': 'star wars'
                    }
                
              }
        }
    },
    'size': 10,
    'explain': True
}
)

resp = requests.get("http://localhost:9200/tmdb/_doc/_search?pretty", 
                   data=search,
                   headers={"Content-Type": "application/json"}).json()

for idx, hit in enumerate(resp['hits']['hits']):
    print(idx, '\t', hit['_score'], '\t', hit['_source']['title'])

0 	 14.902508 	 Star Wars: Episode VI - Return of the Jedi
1 	 11.558649 	 Star Wars: Episode IV - A New Hope
2 	 11.558649 	 Star Wars: Episode I - The Phantom Menace
3 	 11.558649 	 Star Wars: Episode III - Revenge of the Sith
4 	 11.558649 	 Star Wars: Episode II - Attack of the Clones
5 	 10.452412 	 Star Wars: Episode V - The Empire Strikes Back


In [45]:
resp

{'error': {'col': 226,
  'line': 1,
  'reason': 'no [query] registered for [query]',
  'root_cause': [{'col': 226,
    'line': 1,
    'reason': 'no [query] registered for [query]',
    'type': 'parsing_exception'}],
  'type': 'parsing_exception'},
 'status': 400}